In [1]:
import os
import sys
from glob import glob

import cartopy.crs as ccrs
import cmocean
import dask
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

# Local Utils
sys.path.insert(0, "/glade/work/zespinosa/Projects/utils")
from utils import (
    plot_stationary_sp,
    xarray_monthly_to_time,
    xarray_time_to_monthly,
)

LOAD DATA

In [2]:
# Load cesm-nudged psl
cesm_nudged_psl_anoms = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_psl_anoms_198501-202412.nc")
cesm_nudged_psl = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_psl_198501-202412.nc")

# Load cesm-nudged siconc
cesm_nudged_siconc_anoms = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_siconc-anoms_198501-202412.nc")
cesm_nudged_siconc = xr.open_dataset("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_siconc_198501-202412.nc")

# Load nsidc siconc
nsidc_siconc_anoms = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/nsidc_siconc-anoms_197901-202211.nc")
nsidc_siconc = xr.open_dataset("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/nsidc_siconc-anoms_197901-202211.nc")

# Load era5 psl
era5_slp = xr.open_dataset("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/era5_slp-monthly_197901-202211.nc")
era5_slp_anoms = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/era5_slp-anoms_197901-202211.nc")

Observations Spatial Plots

In [3]:
def spatial_obs_anoms_plots(siconc, psl, title, sp):
    fig, ax = plot_stationary_sp()
    # Plot siconc filled contours
    img = ax.contourf(
        siconc.longitude,
        siconc.latitude,
        siconc,
        transform=ccrs.SouthPolarStereo(), 
        levels=np.arange(-.9, 1.1, .2),
        cmap="RdBu",
    )
    # Plot Surface Pressure Contours
    con = ax.contour(
        psl.longitude,
        psl.latitude,
        psl,
        transform=ccrs.PlateCarree(), 
        levels=np.arange(-14, 16, 3),
        colors=["black"],
        negative_linestyle="dashed",
    )
    ax.clabel(con, con.levels, inline=True, fontsize=14)
    cbar2 = fig.colorbar(img, ax=ax)
    ax.set_title(title)
    fig.set_size_inches(8, 8)
    plt.savefig(sp)
    plt.clf()

def create_monthly_obs_plots(siconc, psl):
    for year in [2021, 2022] : 
        for month in range(1, 13):
            cur_siconc = siconc.sel(year=year, month=month)
            cur_psl = psl.sel(year=year, month=month)
            spatial_obs_anoms_plots(
                siconc=cur_siconc,
                psl=cur_psl/100, # Convert from Pa to hPa 
                title=f"Observations {year}-{month}",
                sp=os.path.join(f"/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/figures/observations_siconc-slp-anoms_{year}-{month}.png")
            )

# create_monthly_obs_plots(nsidc_siconc_anoms, era5_slp_anoms)

In [ ]:
CESM1-Nudged Spatial Plots

In [6]:
def spatial_cesm_anoms_plots(siconc, psl, title, sp):
    fig, ax = plot_stationary_sp()
    # Plot siconc filled contours
    img = ax.contourf(
        siconc.TLON,
        siconc.TLAT,
        siconc,
        transform=ccrs.PlateCarree(), 
        levels=np.arange(-.9, 1.1, .2),
        cmap="RdBu",
    )
    # Plot Surface Pressure Contours
    con = ax.contour(
        psl.lon,
        psl.lat,
        psl,
        transform=ccrs.PlateCarree(), 
        levels=np.arange(-14, 16, 3),
        colors=["black"],
        negative_linestyle="dashed",
    )
    ax.clabel(con, con.levels, inline=True, fontsize=14)
    cbar2 = fig.colorbar(img, ax=ax)
    ax.set_title(title)
    fig.set_size_inches(8, 8)
    plt.savefig(sp)
    plt.clf()

def create_monthly_cesm_plots(siconc, psl):
    for year in [2021, 2022] : 
        for month in range(1, 13):
            cur_siconc = siconc.sel(year=year, month=month)
            cur_psl = psl.sel(year=year, month=month)
            spatial_cesm_anoms_plots(
                siconc=cur_siconc/100, # Convert to -1 to 1
                psl=cur_psl/100, # Convert from Pa to hPa 
                title=f"CESM-Nudged {year}-{month}",
                sp=os.path.join(f"/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/figures/cesm-nudged_siconc-slp-anoms_{year}-{month}.png")
            )

create_monthly_cesm_plots(siconc=cesm_nudged_siconc_anoms, psl=cesm_nudged_psl_anoms)

/glade/work/zespinosa/conda-envs/cenv/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1666: UserWarning: The following kwargs were not used by contour: 'negative_linestyle'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)
/glade/work/zespinosa/Projects/utils/utils.py:88: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=[20, 10])


<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

...

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

In [ ]:
def spatial_plot(siconc, title, lon, lat, levels=np.arange(-100, 110, 10), cmap="RdBu"):
    fig, ax = plot_stationary_sp()
    img = ax.contourf(
        lon,
        lat,
        siconc,
        transform=ccrs.PlateCarree(), 
        levels=levels,
        # vmin=-0.90,
        # vmax=0.90,
        cmap=cmap,
    )
    cbar2 = fig.colorbar(img, ax=ax)
    ax.set_title(title)
    fig.set_size_inches(6, 6)
    
def spatial_plot_atm(siconc, title, lon, lat, levels=np.arange(-14, 16, 2), cmap="RdBu"):
    fig, ax = plot_stationary_sp()
    img = ax.contour(
        lon,
        lat,
        siconc,
        transform=ccrs.PlateCarree(), 
        levels=levels,
        colors=["black"],
        negative_linestyle="dashed",
    )
    ax.clabel(img, img.levels, inline=True, fontsize=14)
    ax.set_title(title)
    fig.set_size_inches(6, 6)